In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [145]:
cols =['DateStart','AdsetId','LinkClick','Conversions','Impressions','Spend']
data = pd.read_csv('RagingBull_FB_Hourly.csv', usecols=cols)
data.to_csv('Final_Set')
data.fillna(0,inplace=True)

In [146]:
data = pd.read_csv('Final_Set')
data.fillna(0, inplace= True)

In [147]:
data['CTR'] = (data['LinkClick']/data['Impressions']).replace([np.inf, -np.inf], 0)

In [148]:
data.head()

,Unnamed: 0,Spend,DateStart,LinkClick,Impressions,AdsetId,Conversions,CTR
0,0,1.58,2018-07-12,0.0,27.0,6094650899637,0.0,0.0
1,1,0.02,2018-05-18,0.0,3.0,6093425908037,0.0,0.0
2,2,0.02,2018-05-26,0.0,2.0,6093224252437,0.0,0.0
3,3,0.04,2018-06-07,0.0,7.0,6094650898437,0.0,0.0
4,4,0.04,2018-06-04,0.0,2.0,6095173565837,0.0,0.0


In [149]:
data_total_IPS = pd.pivot_table(data = data, values= ['Spend','Impressions'], fill_value= 0, index='AdsetId',aggfunc='sum')

In [151]:
data_total_IPS['IPS'] = (data_total_IPS['Impressions']/ data_total_IPS['Spend']).replace([np.inf, -np.inf], 0)

In [154]:
data_total_IPS.drop(['Impressions','Spend'], axis=1, inplace=True)

In [158]:
ads_IPS = data_total_IPS.values

In [57]:
data_total_CTR = pd.pivot_table(data = data, values= ['LinkClick','Impressions'], fill_value= 0, index='AdsetId',aggfunc='sum')

In [58]:
data_total_CTR['CTR_agg'] = (data_total_CTR['LinkClick']/data_total_CTR['Impressions']).replace([np.inf, -np.inf], 0)
data_total_CTR.drop(['LinkClick','Impressions'],inplace = True, axis = 1)
data_first_row = pd.pivot_table(data = data_total_CTR, values= ['CTR_agg'],columns='AdsetId').replace([np.inf, -np.inf], 0)

In [59]:
data_first_row.head()

AdsetId,6082903610237,6082903612037,6082903614437,6082903615637,6082903618837,6082903621237,6088935250037,6088935251437,6088935252837,6089948390437,...,23842904874210027,23842974432060616,23842974432080616,23842974432090616,23842974432100616,23842974432160616,23842974432170616,23843001848720616,23843001951860616,23843001969560616
CTR_agg,0.013802,0.01367,0.010551,0.009154,0.012005,0.013056,0.009543,0.011484,0.010145,0.01272,...,0.013622,0.008832,0.012568,0.014083,0.0051,0.009281,0.009237,0.005055,0.004031,0.005489


In [60]:
data_total_CTR.head()

,CTR_agg
AdsetId,
6082903610237,0.013802
6082903612037,0.013670
6082903614437,0.010551
6082903615637,0.009154
6082903618837,0.012005


In [61]:
data = data.join(data_total_CTR, on = ['AdsetId'])

In [62]:
data = data[data['CTR_agg'] > 0]

In [63]:
data.drop(['CTR_agg'], inplace= True, axis = 1)

In [64]:
data.head()

,Unnamed: 0,DateStart,LinkClick,Impressions,AdsetId,Conversions,CTR
0,0,2018-07-12,0.0,27.0,6094650899637,0.0,0.0
1,1,2018-05-18,0.0,3.0,6093425908037,0.0,0.0
2,2,2018-05-26,0.0,2.0,6093224252437,0.0,0.0
3,3,2018-06-07,0.0,7.0,6094650898437,0.0,0.0
4,4,2018-06-04,0.0,2.0,6095173565837,0.0,0.0


In [65]:
data.shape

(438872, 7)

In [66]:
len(set(data['AdsetId']))

242

In [67]:
opt_data = (data[['DateStart','AdsetId','CTR']].copy(deep = True))

In [68]:
data_final = pd.pivot_table(data=opt_data, values= 'CTR', columns='AdsetId',fill_value= 0,index='DateStart',aggfunc='sum').reset_index()
data_final.drop('DateStart',inplace= True, axis=1)
data_final.shape

(130, 242)

In [69]:
# data_initial_set = pd.pivot_table(data = data, values= ['LinkClick','conversions'], fill_value= 0, index='id',aggfunc='sum')

In [70]:
# data_initial_set['TotalClicksperConversion'] = (data_initial_set['LinkClick']/data_initial_set['Conversions']).replace([np.inf, -np.inf,np.NAN], 0)

In [71]:
# data_initial_set = pd.pivot_table(data = data_initial_set, columns= 'id', values= 'TotalClicksperConversion', fill_value= 0).reset_index()

In [72]:
# data_initial_set.drop('index',inplace= True,axis =1)

In [73]:
# data_initial_set

In [74]:
# import gc
# del opt_data
# # del raw_data
# gc.collect()

In [75]:
len_data_final = data_final.shape[1]
df = pd.DataFrame()
appender = data_first_row.iloc[[0]]
while len_data_final >= 0:
    df = df.append(appender,ignore_index= True)
    len_data_final -= 1
df.shape 

(243, 242)

In [76]:
data_final = df.append(data_final)
data_final.shape

(373, 242)

In [77]:
import random
import math
from collections import defaultdict
N = data_final.shape[0]
d = data_final.shape[1]
Exp = 1000 ###Number of experiments
experiment_data = defaultdict()
for exp in range(Exp):
    id_reward_map = defaultdict()
    id_reward_map = {}
    numbers_of_rewards_1 = [0] * d
    numbers_of_rewards_0 = [0] * d
    sums_of_rewards = [0] * d
    total_reward = 0
    reward = 0
    for key, value in enumerate(data_final.columns):
        id_reward_map[key] = [0, value, 0]

    for n in range(0, N):
        ad = 0
        max_random = 0
        for i in range(0, d):
            random_beta = random.betavariate(numbers_of_rewards_1[i] + 1, numbers_of_rewards_0[i] + 1)
    #         print("Ad id {}  Max random .{}  Random beta .{}".format(i, max_random, random_beta))
    #         print("".format())
            if random_beta > max_random:
                max_random = random_beta
                ad = i
    #             print(ad)
                id_reward_map[ad][0] += 1
        #     print(ads_selected)
                reward = data_final.values[n, ad]
                if reward > 0:
                    numbers_of_rewards_1[ad]+= 1
                    id_reward_map[ad][2]+=reward
                    sums_of_rewards[ad] = sums_of_rewards[ad] + reward
                    total_reward += reward
                else:
                    numbers_of_rewards_0[ad]+= 1
#             print("Reward {}".format(reward))
#             print("Total re")
                
    experiment_data[exp] = {'id_reward_map': id_reward_map, 'total_reward': total_reward}

    #Visualising the results
    # plt.hist(id_reward_map[1])
    # plt.title('Histogram of ads selections')
    # plt.xlabel('Ads')
    # plt.ylabel('Number of times each ad was selected')
    # plt.show()

In [85]:
val = sorted([(value['total_reward'],key) for (key,value) in experiment_data.items()], reverse=True)[:1]
val
total_weights = val[0][0]
experiment_selected = val[0][1]

In [114]:
# Weights_selected = 
ad_weights = defaultdict()
for key, value in experiment_data.items():
    for k,val in value['id_reward_map'].items():
        ad_weights[val[1]] = (val[2], val[2]/total_reward)
        

In [108]:
# sum(ad_weights.values())

In [109]:
print(sum(x[1] for x in ad_weights.values()))

1.0000000000000002


In [110]:
total_reward

209.3906160292191

In [125]:
ad_weights

defaultdict(None,
            {6082903610237: (4.7682042539023115, 0.02277181444099166),
             6082903612037: (0.5467895641306048, 0.002611337482546514),
             6082903614437: (0.274335804339592, 0.001310162840828121),
             6082903615637: (0.10984516505988896, 0.0005245944978000389),
             6082903618837: (0.389878717078887, 0.001861968432360321),
             6082903621237: (0.261116367076632, 0.0012470299387255964),
             6088935250037: (0.1049746192893401, 0.0005013339245092606),
             6088935251437: (0.1378043178686265, 0.0006581207910931252),
             6088935252837: (0.030433844161450466, 0.00014534483320495902),
             6089948390437: (0.012720156555772993, 6.074845567099329e-05),
             6089948393437: (0.13288780362421285, 0.0006346406832561648),
             6093223857637: (0.018245768300725433, 8.713746894072538e-05),
             6093223982237: (19.320053376097917, 0.09226800007791146),
             6093224252437: (4.258

In [93]:
import scipy
from scipy.optimize import linprog

In [117]:
ads = []
ads_CTR = []
ad_weight_ratio = []
for key, value in ad_weights.items():
    ads.append(key)
    ads_CTR.append(value[0])
    ad_weight_ratio.append(value[1])

In [122]:
ads = np.array(ads)
ads_CTR = np.array(-1 * ads_CTR)
ad_weight_ratio = np.array(ad_weight_ratio)

In [126]:
data.head()

,Unnamed: 0,DateStart,LinkClick,Impressions,AdsetId,Conversions,CTR
0,0,2018-07-12,0.0,27.0,6094650899637,0.0,0.0
1,1,2018-05-18,0.0,3.0,6093425908037,0.0,0.0
2,2,2018-05-26,0.0,2.0,6093224252437,0.0,0.0
3,3,2018-06-07,0.0,7.0,6094650898437,0.0,0.0
4,4,2018-06-04,0.0,2.0,6095173565837,0.0,0.0


In [123]:
ads_CTR

(242,)

In [159]:
IPS=np.zeros((len(set(data['AdsetId'])),len(set(data['AdsetId']))))
np.fill_diagonal(IPS,ads_IPS*(-1))

In [160]:
IPS

array([[-37.41715923,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        , -37.9813683 ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , -34.35704858, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ..., -63.63392666,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
        -65.65228897,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        , -53.54954416]])

In [161]:
budget = np.ones_like(ads_CTR)

In [163]:
len(budget)

242

In [257]:
A=np.vstack((budget,ad_weight_ratio,IPS))

In [258]:
c = ((-1) * ads_CTR).reshape(-1,1)

In [331]:
total_budget = 100000
second_budget = 10
b = np.hstack((total_budget, second_budget, ([-100] * len(budget)))).reshape(-1,1)
bounds = [(None, None) for x in range(len(budget))]

In [332]:
res = linprog(c,A_ub=A,b_ub=b,bounds=bounds,method='interior-point',options={"disp": True})

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 759.9155622302      
0.3863755414826     0.3863755414826     0.3863755414969     0.6265433889136  0.3863755414827     973.5487460326      
0.1296318092098     0.1296318092098     0.1296318092103     0.7063476843631  0.1296318092098     974.1673387926      
0.01752971576716    0.01752971576716    0.017529715767      0.8763763647253  0.01752971576716    1069.263685284      
0.00122260479656    0.00122260479656    0.001222604796552   0.930797642727   0.00122260479656    1661.049633592      
4.262793501642e-05  4.26279350167e-05   4.26279350164e-05   0.969213485771   4.262793501671e-05  2041.180040817      
3.091920842147e-05  3.091920842167e-05  3.091920842145e-05  0.3035426704615  3.091920842168e-05  614.028222365       
3.47170571595e-07   3.471705716068e-07  3.471705716025e-0

In [333]:
print(sum(res.x))

99072.47845831058
